In [113]:
import re #to split the text file
import random #to choose better
import requests #to send the cards to anki
from concurrent.futures import ThreadPoolExecutor #to speed up sending the cards to anki

# Read the content of the file
file_path = "test your understanding.txt" #TODO you might have to change this
with open(file_path, "r") as file:
    content = file.read()

# Split the content at "Test your understanding"
substrings = content.split("Test your understanding")

# Initialize a nested list to store non-empty lines after each substring
nested_list = []

#split before new task
pattern = re.compile(r'\n(?=.{1,2}\))')

# Process each substring and split non-empty lines
for substring in substrings[1:]:  # Start from index 1 to skip the first empty substring
    lines = re.split(pattern, substring)
    nested_list.append(lines[1:]) #first line is also empty

In [114]:
file_path = "test your understanding answers leon.txt" #TODO you might have to change this
# If you put answers in the answer file, use the format "<tut number> <question id (e.g. 'j' or '10')>) 'answer'"

with open(file_path, "r") as file:
    answers_unsplit = file.read()

pattern_answer_lines = re.compile(r'\n(?=\d{1,2} .{1,2}\))')
lines_answers = re.split(pattern_answer_lines, answers_unsplit)

answer_dict = {}
for line in lines_answers:
    answer = re.findall(pattern= r'^(\d{1,2})\s(.{1,2}\))\s(.+)', string = line)
    tut_num = int(answer[0][0]) 
    question_num = answer[0][1]
    answer_text = "Leon- " + answer[0][2]
    if tut_num not in answer_dict:
        answer_dict[tut_num] = {question_num:answer_text}
    else: 
        if question_num not in answer_dict[tut_num]:
            answer_dict[tut_num][question_num] = answer_text
        else:
            answer_dict[tut_num][question_num] = answer_dict[tut_num][question_num] + "\n" + answer_text

#function to find answer
def find_answer(question, tut_num):
    question_num = re.findall(pattern = r"^.{1,2}\)", string = question)[0]
    answer = "Didn't answer yet"
    if tut_num in answer_dict.keys():
        if question_num in answer_dict[tut_num].keys():
            answer = answer_dict[tut_num][question_num]
    return(answer)

In [115]:
# Draw one tut at random from the nested list
random_tut = random.choice(nested_list)
# Identify which element in the outer list it is from
# index_of_random_element = nested_list.index(random_tut)
tut_num = index_of_random_element+1
# choose a random question from the tut
question = random.choice(random_tut)

#print
print(f"Tut: {tut_num}")
print(question)
print(f"Answer: {find_answer(question, tut_num)}")

Tut: 4
a) What is a matrix decomposition?
Answer: Didn't answer yet


In [116]:

#TODO make shure Anki is installed and currently running on the machine
#TODO make shure anki-connect is installed https://github.com/FooSoft/anki-connect
#TODO make shure you have created an deck with the appropriate deck name 

#put all in Anki list without Answer
anki_list = [{"note_type": "Basic",
               "fields": {"Front":f"Tut {tut_i+1}\n{q}",
                          "Back": find_answer(q, tut_i+1)},
                "tags": [f"Tut_{tut_i+1}"]} for tut_i, tut in enumerate(nested_list) for q in tut]

def create_anki_card(payload):
    anki_url = "http://localhost:8765"  # AnkiConnect default URL
    response = requests.post(anki_url, json=payload)
    return response.json()

def create_anki_cards(cards_data):
    deck_name = "ML 2023 Test your understanding"  #TODO Change this to your deck name
    with ThreadPoolExecutor() as executor:
        payloads = []
        for card in cards_data:
            tags = ["ML", "Data_Science"]
            note_type = card["note_type"]
            fields = card["fields"]
            tags.extend(card["tags"])

            # Construct the payload for creating a new note
            payload = {
                "action": "addNote",
                "version": 6,
                "params": {
                    "note": {
                        "deckName": deck_name,
                        "modelName": note_type,
                        "fields": fields,
                        "tags": tags,  # Add tags if needed
                    }
                }
            }
            payloads.append(payload)
        # Use ThreadPoolExecutor to parallelize the requests
        responses = list(executor.map(create_anki_card, payloads))

        # Print the responses (optional)
        for response in responses:
            print(response)

create_anki_cards(anki_list) #Took half a minute on my machine

{'result': 1702036978208, 'error': None}
{'result': 1702036978186, 'error': None}
{'result': 1702036978268, 'error': None}
{'result': 1702036978303, 'error': None}
{'result': 1702036978235, 'error': None}
{'result': 1702036978334, 'error': None}
{'result': 1702036978747, 'error': None}
{'result': 1702036978773, 'error': None}
{'result': 1702036978362, 'error': None}
{'result': 1702036978796, 'error': None}
{'result': 1702036980279, 'error': None}
{'result': 1702036980314, 'error': None}
{'result': 1702036980359, 'error': None}
{'result': 1702036980396, 'error': None}
{'result': 1702036980429, 'error': None}
{'result': 1702036980462, 'error': None}
{'result': 1702036980495, 'error': None}
{'result': 1702036980846, 'error': None}
{'result': 1702036980879, 'error': None}
{'result': 1702036980912, 'error': None}
{'result': 1702036982346, 'error': None}
{'result': 1702036982401, 'error': None}
{'result': 1702036982432, 'error': None}
{'result': 1702036982463, 'error': None}
{'result': 17020